# **Recomendation System using cosine similiraty**

**Imports**

In [1]:
import pandas as pd

**Dataset**

In [3]:
data=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/project/ML/recommendation system/spotify_dataset.csv')
data.head()

,Index,Highest Charting Position,Number of Times Charted,Week of Highest Charting,Song Name,Streams,Artist,Artist Followers,Song ID,Genre,...,Danceability,Energy,Loudness,Speechiness,Acousticness,Liveness,Tempo,Duration (ms),Valence,Chord
0,1,1,8,2021-07-23--2021-07-30,Beggin',"48,633,449",Måneskin,3377762,3Wrjm47oTz2sjIgck11l5e,"['indie rock italiano', 'italian pop']",...,0.714,0.8,-4.808,0.0504,0.127,0.359,134.002,211560,0.589,B
1,2,2,3,2021-07-23--2021-07-30,STAY (with Justin Bieber),"47,248,719",The Kid LAROI,2230022,5HCyWlXZPP0y6Gqq8TgA20,['australian hip hop'],...,0.591,0.764,-5.484,0.0483,0.0383,0.103,169.928,141806,0.478,C#/Db
2,3,1,11,2021-06-25--2021-07-02,good 4 u,"40,162,559",Olivia Rodrigo,6266514,4ZtFanR9U6ndgddUvNcjcG,['pop'],...,0.563,0.664,-5.044,0.154,0.335,0.0849,166.928,178147,0.688,A
3,4,3,5,2021-07-02--2021-07-09,Bad Habits,"37,799,456",Ed Sheeran,83293380,6PQ88X9TkUIAUIZJHW2upE,"['pop', 'uk pop']",...,0.808,0.897,-3.712,0.0348,0.0469,0.364,126.026,231041,0.591,B
4,5,5,1,2021-07-23--2021-07-30,INDUSTRY BABY (feat. Jack Harlow),"33,948,454",Lil Nas X,5473565,27NovPIUIRrOZoCHxABJwK,"['lgbtq+ hip hop', 'pop rap']",...,0.736,0.704,-7.409,0.0615,0.0203,0.0501,149.995,212000,0.894,D#/Eb


In [4]:
data.isnull().sum()

Index                        0
Highest Charting Position    0
Number of Times Charted      0
Week of Highest Charting     0
Song Name                    0
Streams                      0
Artist                       0
Artist Followers             0
Song ID                      0
Genre                        0
Release Date                 0
Weeks Charted                0
Popularity                   0
Danceability                 0
Energy                       0
Loudness                     0
Speechiness                  0
Acousticness                 0
Liveness                     0
Tempo                        0
Duration (ms)                0
Valence                      0
Chord                        0
dtype: int64

**Feature extraction**

function for take important features to a single column

In [5]:
def important_features(dataset):
    data=dataset.copy()
    for i in range(0, dataset.shape[0]):
        data["imp"]=data["Song Name"]+' '+data["Artist"]+" "+data["Genre"]
    return data


In [6]:
data=important_features(data)
data.head(2)

,Index,Highest Charting Position,Number of Times Charted,Week of Highest Charting,Song Name,Streams,Artist,Artist Followers,Song ID,Genre,...,Energy,Loudness,Speechiness,Acousticness,Liveness,Tempo,Duration (ms),Valence,Chord,imp
0,1,1,8,2021-07-23--2021-07-30,Beggin',"48,633,449",Måneskin,3377762,3Wrjm47oTz2sjIgck11l5e,"['indie rock italiano', 'italian pop']",...,0.8,-4.808,0.0504,0.127,0.359,134.002,211560,0.589,B,"Beggin' Måneskin ['indie rock italiano', 'ital..."
1,2,2,3,2021-07-23--2021-07-30,STAY (with Justin Bieber),"47,248,719",The Kid LAROI,2230022,5HCyWlXZPP0y6Gqq8TgA20,['australian hip hop'],...,0.764,-5.484,0.0483,0.0383,0.103,169.928,141806,0.478,C#/Db,STAY (with Justin Bieber) The Kid LAROI ['aust...


providing index to dataset

In [7]:
data["ids"]=[i for i in range(0,data.shape[0])]

**Tfidf Vectorizer**

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
vec=TfidfVectorizer()

In [9]:
vecs=vec.fit_transform(data["imp"].apply(lambda x: np.str_(x)))

In [10]:
data.shape , vecs.shape

((1556, 25), (1556, 3088))

**cosine_similarity**

In [11]:
from sklearn.metrics.pairwise import cosine_similarity

In [12]:
sim=cosine_similarity(vecs) 

In [13]:
sim.shape##Columns represnt song-ids and not rows

(1556, 1556)

In [14]:
sim[100][100]

1.0000000000000002

**Prediction**

In [15]:
def recommend_songs(title):
    movie_id=data[data['Song Name']==title]["ids"].values[0]
    scores=list(enumerate(sim[movie_id]))
    sorted_scores=sorted(scores,key=lambda x:x[1],reverse=True)
    sorted_scores=sorted_scores[1:]
    movies=[data[movies[0]==data["ids"]]["Song Name"].values[0] for movies in sorted_scores]
    return movies[0:10]

In [16]:
recommend_songs("Old Town Road")

['Old Town Road - Remix',
 'HOLIDAY',
 'Panini',
 'SUN GOES DOWN',
 'INDUSTRY BABY (feat. Jack Harlow)',
 'MONTERO (Call Me By Your Name)',
 'Star Shopping',
 'Narrow Road (feat. Lil Baby)',
 'p r i d e . i s . t h e . d e v i l (with Lil Baby)',
 'EVERY CHANCE I GET (feat. Lil Baby & Lil Durk)']

**Name of all songs used**

In [17]:
song_name=data['Song Name'].unique()
for i in song_name:
    print(i)


Beggin'
STAY (with Justin Bieber)
good 4 u
Bad Habits
INDUSTRY BABY (feat. Jack Harlow)
MONTERO (Call Me By Your Name)
Kiss Me More (feat. SZA)
Todo De Ti
Yonaguni
I WANNA BE YOUR SLAVE
Levitating (feat. DaBaby)
Qué Más Pues?
Permission to Dance
Peaches (feat. Daniel Caesar & Giveon)
Butter
traitor
deja vu
Save Your Tears (with Ariana Grande) (Remix)
AM Remix
Need To Know
Volando - Remix
Ain't Shit
drivers license
Heat Waves
Blinding Lights
happier
Pepas
Motley Crew
Friday (feat. Mufasa & Hypeman) - Dopamine Re-Edit
Astronaut In The Ocean
Leave The Door Open
Don't Go Yet
favorite crime
Pareja Del Año
Watermelon Sugar
NOT SOBER (feat. Polo G & Stunna Gambino)
RAPSTAR
You Right
911 - Remix
DÁKITI
Miénteme
jealousy, jealousy
Poblado - Remix
Dynamite
The Business
Heartbreak Anthem (with David Guetta & Little Mix)
Mood (feat. iann dior)
Entre Nosotros
Fiel - Remix
Build a Bitch
Nicky Jam: Bzrp Music Sessions, Vol. 41
Tiroteo - Remix
NDA
Rasputin
Leave Before You Love Me (with Jonas Brothers

**Saving the model**

In [19]:
import joblib
joblib.dump(sim,'rec2_model')

['rec2_model']

**Reloding the model**

In [20]:
joblib_reloaded = joblib.load('/content/drive/MyDrive/Colab Notebooks/project/ML/recommendation system/rec2_model') 


In [21]:
def recommend(title):
    movie_id=data[data['Song Name']==title]["ids"].values[0]
    scores=list(enumerate(joblib_reloaded[movie_id]))
    sorted_scores=sorted(scores,key=lambda x:x[1],reverse=True)
    sorted_scores=sorted_scores[1:]
    movies=[data[movies[0]==data["ids"]]["Song Name"].values[0] for movies in sorted_scores]
    return movies[0:10]

In [22]:
recommend('Old Town Road')

['Old Town Road - Remix',
 'HOLIDAY',
 'Panini',
 'SUN GOES DOWN',
 'INDUSTRY BABY (feat. Jack Harlow)',
 'MONTERO (Call Me By Your Name)',
 'Star Shopping',
 'Narrow Road (feat. Lil Baby)',
 'p r i d e . i s . t h e . d e v i l (with Lil Baby)',
 'EVERY CHANCE I GET (feat. Lil Baby & Lil Durk)']